In [4]:
import os
import sys

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

In [2]:
CUDA = True
DATA_PATH = 'Anime dataset'
OUT_PATH = 'output'
LOG_FILE = os.path.join(OUT_PATH, 'log.txt')
BATCH_SIZE = 128
IMAGE_CHANNEL = 3
Z_DIM = 100
G_HIDDEN = 64
X_DIM = 64
D_HIDDEN = 64
EPOCH_NUM = 50
REAL_LABEL = 1
FAKE_LABEL = 0
lr = 2e-4
seed = 1

In [ ]:
np.random.seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device('cuda')

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(Z_DIM, G_HIDDEN * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 8),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(G_HIDDEN * 8, G_HIDDEN * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 4),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(G_HIDDEN * 4, G_HIDDEN * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 2),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(G_HIDDEN * 2, G_HIDDEN, 4, 2, 1, bias=False),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(G_HIDDEN, IMAGE_CHANNEL, 4, 2, 1, bias=False),
            nn.Tanh()
        )
    
    def forward(self, input):
        return self.main(input)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weights.data.normal_(0.0,0.02)
    elif classname.find('BatchNorm') != -1:
        m.weights.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
netG = Generator().to(device)
netG.apply(weighs_init)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(IMAGE_CHANNEL, D_HIDDEN, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(D_HIDDEN, D_HIDDEN * 2, D_HIDDEN, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 2)
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(D_HIDDEN *2, D_HIDDEN * 4, D_HIDDEN, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 4)
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(D_HIDDEN, D_HIDDEN * 4, D_HIDDEN, 8, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 8)
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(D_HIDDEN * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    
    def forward(input):
        return self.main(input).view(-1, 1).squeeze(1)

In [ ]:
netD = Discriminator().to(device)
netD.apply(weights_init)